In [1]:
import os
import sys

seed = 0
os.environ['PYTHONHASHSEED'] = str(seed)

In [2]:
import numpy as np
import pandas as pd
import time
import random

from tensorflow.random import set_seed

In [3]:
# Add modules path
module_path = os.path.abspath('..')
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
from src.data_reader import DataReader
from src.training import train_evaluate_mlp, MLPConfigParams
from src.utils import numbers_list_to_string

In [5]:
# Set seed
random.seed(seed)
np.random.seed(seed)
set_seed(seed)

In [6]:
# Set input and output paths
filename = '../data/N-CMAPSS_DS02-006.h5'
output_path = '../results/new_experiment'

if not os.path.exists(output_path):
    os.makedirs(output_path)

In [7]:
data_reader = DataReader()

start_time = time.process_time()  
data_reader.load_dataset(filename, load_train=True, load_test=True)
print("Operation time (sec): " , (time.process_time() - start_time))

if data_reader.train_set is not None:
    print("Train set shape: " + str(data_reader.train_set.shape))
    
if data_reader.test_set is not None:   
    print("Test set shape: " + str(data_reader.test_set.shape))

Operation time (sec):  3.78125
Train set shape: (5263447, 47)
Test set shape: (1253743, 47)


In [8]:
##################
# Data preparation
##################
train_set = data_reader.train_set
test_set = data_reader.test_set

train_set = train_set.astype(np.float32)
test_set = test_set.astype(np.float32)

y_train = train_set['RUL']
x_train = train_set.drop(['RUL'], axis=1)

selected_columns = data_reader.column_names.w_cols + data_reader.column_names.x_s_cols
x_train = x_train[selected_columns]

y_test = test_set['RUL']
x_test = test_set[x_train.columns]

In [ ]:
###############################
# Model training and evaluation
###############################
NUM_TRIALS = 3
EPOCHS = 60
BATCH_SIZE = 512

mlp_params = MLPConfigParams(layer_sizes=[256, 256, 512, 64], activation='tanh', dropout=0.0)

results_file = os.path.join(output_path, "eval_results.csv")
with open(results_file, "w") as file:
    file.write("mse,rmse,cmapss,mse(mean),mse(std),rmse(mean),rmse(std),cmapss(mean),cmapss(std)\n")

results_path = output_path

mse_vals, rmse_vals, cmapss_vals = train_evaluate_mlp(
    x_train, y_train, 
    x_test, y_test,
    NUM_TRIALS, 
    mlp_params, 
    results_path, 
    EPOCHS, BATCH_SIZE,
    results_file=results_file)

Saved object to file: ../results/new_experiment\split_0\scaler.pkl
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               4864      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 235,137
Trainable params: 235,137
Non-trainable params: 0
_________________________________________________________________
Epoch 1/60
9253/9253 [===========================